# Setup
(No need to read)

In [1]:
# Janky code to do different setup when run in a Colab notebook vs VSCode
DEBUG_MODE = False
try:
    import google.colab
    IN_COLAB = True
    print("Running as a Colab notebook")
    %pip install git+https://github.com/neelnanda-io/TransformerLens.git
    # Install another version of node that makes PySvelte work way faster
    !curl -fsSL https://deb.nodesource.com/setup_16.x | sudo -E bash -; sudo apt-get install -y nodejs
    %pip install git+https://github.com/neelnanda-io/PySvelte.git
except:
    IN_COLAB = False
    print("Running as a Jupyter notebook - intended for development only!")
    from IPython import get_ipython

    ipython = get_ipython()
    # Code to automatically update the HookedTransformer code as its edited without restarting the kernel
    ipython.magic("load_ext autoreload")
    ipython.magic("autoreload 2")

Running as a Colab notebook
  Cloning https://github.com/neelnanda-io/TransformerLens.git to /tmp/pip-req-build-yxiqycbp
  Running command git clone --filter=blob:none --quiet https://github.com/neelnanda-io/TransformerLens.git /tmp/pip-req-build-yxiqycbp
  Resolved https://github.com/neelnanda-io/TransformerLens.git to commit 218ebd6f491f47f5e2f64e4c4327548b60a093eb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 50.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 66.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━


## Installing the NodeSource Node.js 16.x repo...


## Populating apt-get cache...

+ apt-get update
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,084 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-

In [2]:
# Plotly needs a different renderer for VSCode/Notebooks vs Colab argh
import plotly.io as pio

if IN_COLAB or not DEBUG_MODE:
    # Thanks to annoying rendering issues, Plotly graphics will either show up in colab OR Vscode depending on the renderer - this is bad for developing demos! Thus creating a debug mode.
    pio.renderers.default = "colab"
else:
    pio.renderers.default = "png"

In [3]:
# Import stuff
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import einops
from fancy_einsum import einsum
import tqdm.notebook as tqdm
import random
from pathlib import Path
import plotly.express as px
from torch.utils.data import DataLoader

from jaxtyping import Float, Int
from typing import List, Union, Optional
from functools import partial
import copy

import itertools
from transformers import AutoModelForCausalLM, AutoConfig, AutoTokenizer
import dataclasses
import datasets
from IPython.display import HTML

In [4]:
import pysvelte

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import HookedTransformer, HookedTransformerConfig, FactoredMatrix, ActivationCache

In [5]:
model = HookedTransformer.from_pretrained("gpt2-small")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


# len 2

In [6]:
example_prompt = "3 4"
example_answer = " 5"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '3', ' 4']
Tokenized answer: [' 5']


Performance on answer token:
Rank: 0        Logit: 12.88 Prob: 18.99% Token: | 5|

Top 0th token. Logit: 12.88 Prob: 18.99% Token: | 5|
Top 1th token. Logit: 12.16 Prob:  9.20% Token: |/|
Top 2th token. Logit: 11.87 Prob:  6.89% Token: | 4|
Top 3th token. Logit: 11.56 Prob:  5.06% Token: |.|
Top 4th token. Logit: 11.29 Prob:  3.85% Token: | 3|
Top 5th token. Logit: 11.09 Prob:  3.16% Token: | 1|
Top 6th token. Logit: 11.06 Prob:  3.05% Token: |
|
Top 7th token. Logit: 10.99 Prob:  2.85% Token: | 6|
Top 8th token. Logit: 10.70 Prob:  2.14% Token: |-|
Top 9th token. Logit: 10.56 Prob:  1.86% Token: | 0|


Ranks of the answer tokens: [(' 5', 0)]

In [9]:
example_prompt = "33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '33', ' 34']
Tokenized answer: [' 35']


Performance on answer token:
Rank: 4        Logit: 10.84 Prob:  1.87% Token: | 35|

Top 0th token. Logit: 12.30 Prob:  8.00% Token: |.|
Top 1th token. Logit: 11.43 Prob:  3.35% Token: |
|
Top 2th token. Logit: 11.26 Prob:  2.83% Token: | 34|
Top 3th token. Logit: 10.87 Prob:  1.92% Token: | 33|
Top 4th token. Logit: 10.84 Prob:  1.87% Token: | 35|
Top 5th token. Logit: 10.80 Prob:  1.78% Token: | 36|
Top 6th token. Logit: 10.60 Prob:  1.46% Token: | 0|
Top 7th token. Logit: 10.52 Prob:  1.34% Token: | 32|
Top 8th token. Logit: 10.47 Prob:  1.28% Token: | 29|
Top 9th token. Logit: 10.36 Prob:  1.15% Token: | 38|


Ranks of the answer tokens: [(' 35', 4)]

In [11]:
example_prompt = "null 33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'null', ' 33', ' 34']
Tokenized answer: [' 35']


Performance on answer token:
Rank: 0        Logit: 16.19 Prob: 68.71% Token: | 35|

Top 0th token. Logit: 16.19 Prob: 68.71% Token: | 35|
Top 1th token. Logit: 12.85 Prob:  2.45% Token: | 33|
Top 2th token. Logit: 12.59 Prob:  1.89% Token: |
|
Top 3th token. Logit: 12.57 Prob:  1.85% Token: | 37|
Top 4th token. Logit: 12.45 Prob:  1.64% Token: | 34|
Top 5th token. Logit: 12.30 Prob:  1.41% Token: | 36|
Top 6th token. Logit: 12.10 Prob:  1.15% Token: | 40|
Top 7th token. Logit: 12.00 Prob:  1.04% Token: | 45|
Top 8th token. Logit: 11.98 Prob:  1.03% Token: | 0|
Top 9th token. Logit: 11.57 Prob:  0.68% Token: | 30|


Ranks of the answer tokens: [(' 35', 0)]

In [10]:
example_prompt = "32 33 34"
example_answer = " 35"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '32', ' 33', ' 34']
Tokenized answer: [' 35']


Performance on answer token:
Rank: 0        Logit: 17.90 Prob: 76.47% Token: | 35|

Top 0th token. Logit: 17.90 Prob: 76.47% Token: | 35|
Top 1th token. Logit: 14.62 Prob:  2.88% Token: |
|
Top 2th token. Logit: 14.52 Prob:  2.60% Token: | 36|
Top 3th token. Logit: 14.29 Prob:  2.07% Token: | 34|
Top 4th token. Logit: 13.95 Prob:  1.47% Token: | 37|
Top 5th token. Logit: 13.75 Prob:  1.20% Token: | 33|
Top 6th token. Logit: 13.50 Prob:  0.94% Token: | 40|
Top 7th token. Logit: 13.42 Prob:  0.87% Token: | 45|
Top 8th token. Logit: 12.86 Prob:  0.49% Token: | 30|
Top 9th token. Logit: 12.67 Prob:  0.41% Token: | 25|


Ranks of the answer tokens: [(' 35', 0)]

In [8]:
example_prompt = "1 2"
example_answer = " 3"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', '1', ' 2']
Tokenized answer: [' 3']


Performance on answer token:
Rank: 0        Logit: 16.69 Prob: 89.34% Token: | 3|

Top 0th token. Logit: 16.69 Prob: 89.34% Token: | 3|
Top 1th token. Logit: 12.56 Prob:  1.44% Token: |
|
Top 2th token. Logit: 11.64 Prob:  0.58% Token: | 4|
Top 3th token. Logit: 11.10 Prob:  0.33% Token: | 1|
Top 4th token. Logit: 10.85 Prob:  0.26% Token: | 5|
Top 5th token. Logit: 10.80 Prob:  0.25% Token: | Next|
Top 6th token. Logit: 10.78 Prob:  0.24% Token: |.|
Top 7th token. Logit: 10.35 Prob:  0.16% Token: | 2|
Top 8th token. Logit: 10.16 Prob:  0.13% Token: | /|
Top 9th token. Logit: 10.11 Prob:  0.12% Token: |/|


Ranks of the answer tokens: [(' 3', 0)]

In [7]:
example_prompt = "March April"
example_answer = " May"
utils.test_prompt(example_prompt, example_answer, model, prepend_bos=True)

Tokenized prompt: ['<|endoftext|>', 'March', ' April']
Tokenized answer: [' May']


Performance on answer token:
Rank: 0        Logit: 22.59 Prob: 96.57% Token: | May|

Top 0th token. Logit: 22.59 Prob: 96.57% Token: | May|
Top 1th token. Logit: 17.70 Prob:  0.73% Token: | July|
Top 2th token. Logit: 17.46 Prob:  0.57% Token: | June|
Top 3th token. Logit: 16.94 Prob:  0.34% Token: | April|
Top 4th token. Logit: 16.85 Prob:  0.31% Token: | September|
Top 5th token. Logit: 16.68 Prob:  0.26% Token: | 5|
Top 6th token. Logit: 16.60 Prob:  0.24% Token: |
|
Top 7th token. Logit: 16.34 Prob:  0.19% Token: | November|
Top 8th token. Logit: 16.20 Prob:  0.16% Token: | March|
Top 9th token. Logit: 15.67 Prob:  0.10% Token: | October|


Ranks of the answer tokens: [(' May', 0)]